# Setup

In [1]:
# do this before anything else because you don't want to have to rerun this 
# every time you need to install something new - pain in the ass

# Install Earth Engine API
!pip install -q earthengine-api 

# import ee
import ee
# from ee.oauth import get_credentials_path
# print(get_credentials_path())

# Authenticate Earth Engine
ee.Authenticate(force=True)
print('GEE authenticated')

Enter verification code:  4/1Ab32j93BTaGZ3zgQ0sda67Bw5TYcjXTDKXyfN9mdW5HcvJ6V8HxOgnJxezs



Successfully saved authorization token.
GEE authenticated


In [ ]:
# Initialize Earth Engine
ee.Initialize(project='columbia-phd')
print('GEE initialized')

# import whatever else you need

# GEEmap for visualization
!pip install -q geemap
# !pip install -q ipywidgets geemap
# !pip install -q ipywidgets geemap jupyterlab_widgets
import geemap
print('GEE imported')

In [ ]:
# import data

# AOI 
# -----------------------------------------------------------------------------------------------------
faoFC = ee.FeatureCollection("FAO/GAUL/2015/level2")
aoiFC = faoFC.filter(ee.Filter.eq('ADM1_NAME', 'Madre de Dios'))
# print(aoiFC)

# MODIS Vegetation Indices
# -----------------------------------------------------------------------------------------------------

# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13A3
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MYD13A3
modisTerraIC = (
    ee.ImageCollection('MODIS/061/MOD13A3')
    .filter(ee.Filter.date('2000-01-01', '2024-12-31'))
    .select(['NDVI', 'EVI', 'SummaryQA'])
    .map(lambda img: img.clip(aoiFC))
)

modisAquaIC = (
    ee.ImageCollection('MODIS/061/MYD13A3')
    .filter(ee.Filter.date('2000-01-01', '2024-12-31'))
    .select(['NDVI', 'EVI', 'SummaryQA'])
    .map(lambda img: img.clip(aoiFC))
)

# print(terraClimIC)

res = modisTerraIC.first().projection().nominalScale().getInfo()

# print pixel size
print('pixel size:', res) 
print('band names:', modisTerraIC.first().bandNames().getInfo())
print("# of images Terra:", modisTerraIC.size().getInfo())
print("# of images Aqua:", modisAquaIC.size().getInfo())

In [ ]:
# cloud mask and combine according to metadata

# define a function that scales each band in an image
def mask_summaryQA(img):
    # Create a mask where SummaryQA == 0
    qa = img.select('SummaryQA')
    qa_mask = qa.eq(0).Or(qa.eq(1))
    
    # Apply the mask to all bands
    return img.updateMask(qa_mask)

# Apply to the collections
modisTerraIC_masked = (modisTerraIC.map(mask_summaryQA))

modisAquaIC_masked = (modisAquaIC.map(mask_summaryQA))

In [8]:
# Create map
m = geemap.Map()
m.centerObject(aoiFC, zoom=7)

# get min and max values for visualization
modisAquaIC_first = modisAquaIC_masked.first()
minMaxStats = modisAquaIC_first.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoiFC,          
    scale=res,              
    maxPixels=1e13
)

# print(minMaxStats.getInfo())
# print(minMaxStats.get('tmmx_min').getInfo())
# print(minMaxStats.get('tmmx_max').getInfo())

# define visualization parameters
palette = [
        'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
        '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
        '012e01', '011d01', '011301'
    ]

# add to map
m.addLayer(aoiFC, {}, "AOI")
m.addLayer(modisAquaIC_first.select('NDVI'), 
           {'min': minMaxStats.get('NDVI_min'),
            'max': minMaxStats.get('NDVI_max'),
            'palette':palette}, 
           "Aqua NDVI")
m.addLayer(modisAquaIC_first.select('EVI'), 
           {'min': minMaxStats.get('EVI_min'),
            'max': minMaxStats.get('EVI_max'),
            'palette':palette}, 
           "Aqua EVI")






# get min and max values for visualization
modisTerraIC_first = modisTerraIC_masked.first()
minMaxStats = modisTerraIC_first.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoiFC,          
    scale=res,              
    maxPixels=1e13
)

# print(minMaxStats.getInfo())
# print(minMaxStats.get('tmmx_min').getInfo())
# print(minMaxStats.get('tmmx_max').getInfo())

# define visualization parameters
palette = [
        'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
        '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
        '012e01', '011d01', '011301'
    ]

# add to map
m.addLayer(modisTerraIC_first.select('NDVI'), 
           {'min': minMaxStats.get('NDVI_min'),
            'max': minMaxStats.get('NDVI_max'),
            'palette':palette}, 
           "Terra NDVI")
m.addLayer(modisTerraIC_first.select('EVI'), 
           {'min': minMaxStats.get('EVI_min'),
            'max': minMaxStats.get('EVI_max'),
            'palette':palette}, 
           "Terra EVI")

m

Map(center=[-11.966886618395746, -70.5430491275674], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
# combine into a single collection
modisIC_combo = modisTerraIC_masked.merge(modisAquaIC_masked)
print("# of images:", modisIC_combo.size().getInfo())

# get monthly mean values

# define a function that adds year and month on as properties
def add_year_month(img):
    date = ee.Date(img.get('system:time_start'))
    year = date.get('year')
    month = date.get('month')
    return img.set({'year': year, 'month': month})

# map the function over the collection
modisIC_withYM = modisIC_combo.map(add_year_month)

# create lists of years and months
years = ee.List.sequence(2000, 2024)
months = ee.List.sequence(1, 12)

modisIC_monthly = ee.ImageCollection(
    ee.List(
        years.map(
            lambda y: ee.List(
                months.map(
                    lambda m: modisIC_withYM
                        .filter(ee.Filter.eq('year', y))
                        .filter(ee.Filter.eq('month', m))
                        .mean()
                        .set('year', y)
                        .set('month', m)
                )
            )
        ).flatten()
    )
)

print("done")

# of images: 569
done


In [12]:
# Create map
m = geemap.Map()
m.centerObject(aoiFC, zoom=7)

# get min and max values for visualization
modisIC_selectedMonth = modisIC_monthly.filter(ee.Filter.date('2010-01-01', '2010-01-31')).first()
# minMaxStats = modisIC_selectedMonth.reduceRegion(
#     reducer=ee.Reducer.minMax(),
#     geometry=aoiFC,          
#     scale=res,              
#     maxPixels=1e13
# )

# define visualization parameters
palette = [
        'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
        '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
        '012e01', '011d01', '011301'
    ]

# add to map
m.addLayer(aoiFC, {}, "AOI")
m.addLayer(modisIC_selectedMonth.select('NDVI'), 
           {'min': 0,
            'max': 9000,
            'palette':palette}, 
           "NDVI")
m.addLayer(modisIC_selectedMonth.select('EVI'), 
           {'min': 0,
            'max': 9000,
            'palette':palette}, 
           "EVI")

m

EEException: User memory limit exceeded.

In [ ]:
# # check dates

# print(terraClimIC_scaled.size().getInfo())

# years = ee.List.sequence(2000, 2024)

# def count_by_year(y):
#     y = ee.Number(y)
#     fc = terraClimIC_scaled.filterDate(
#         ee.Date.fromYMD(y, 1, 1),
#         ee.Date.fromYMD(y.add(1), 1, 1)
#     )
#     return ee.Feature(None, {
#         'year': y,
#         'count': fc.size()
#     })

# summary = ee.FeatureCollection(years.map(count_by_year))
# # print(summary.getInfo())

# simple = summary.aggregate_array('year').zip(
#     summary.aggregate_array('count')
# ).getInfo()

# for yr, cnt in simple:
#     print(yr, cnt)



In [ ]:
# export the gee image to the python environment
# STUCK ON EXPORTING IC?